<a href="https://colab.research.google.com/github/AroubAlRizq/advanced_ai_exercises/blob/main/Lab2SOLUTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Separate imports for tidiness

In [10]:
import torch, torchvision, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import torch

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
transform=transforms.Compose([transforms.ToTensor() ,
transforms.Normalize((0.5,), (0.5,))])  # BUG normalize DONE

loader=DataLoader(torchvision.datasets.MNIST('./data',download=True,train = True,transform=transform),batch_size=256,shuffle=True,num_workers=2,pin_memory=True)
z_dim=100; g_lr=2e-2; d_lr=2e-5

In [8]:
batch = next(iter(loader))

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [9]:
batch[0].shape

torch.Size([256, 1, 28, 28])

In [12]:
class D(nn.Module):
    def __init__(self):
        super().__init__();
        self.net=nn.Sequential(nn.Conv2d(1,32,4,2,1),
                               nn.LeakyReLU(0.2,True),
                               nn.Conv2d(32,64,4,2,1),
                               nn.BatchNorm2d(64),
                               nn.LeakyReLU(0.2,True),
                               nn.Conv2d(64,1,7,1,0),
                               nn.Sigmoid())

    def forward(self,x):
      return self.net(x).view(x.size(0),1)


In [20]:
class G(nn.Module):
    def __init__(self):
        super().__init__();
        self.net=nn.Sequential(
            nn.ConvTranspose2d(100,128,7,1,0),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128,64,4,2,1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64,1,4,2,1),
            nn.Tanh())
             # BUG missing Tanh DONE

    def forward(self,z):
      z=z.view(z.size(0),100,1,1);
      return self.net(z)  # BUG reshape

Dnet=D().to(device);
Gnet=G().to(device)
crit=nn.BCEWithLogitsLoss();
opt_d=torch.optim.Adam(Dnet.parameters(),
                       lr=d_lr,betas=(0.9,0.999)); opt_g=torch.optim.Adam(Gnet.parameters(),
                                                                          lr=g_lr,betas=(0.9,0.999))


In [21]:
from tqdm import tqdm
for real,_ in tqdm(loader):
    real=real.to(device); b=real.size(0)
    z=torch.randn(b,z_dim,device=device);
    fake=Gnet(z.view(b,z_dim,1,1))
    loss_d=crit(Dnet(real),
                torch.ones(b,1,device=device)) # D(x) should be 1
    +crit(Dnet(fake),torch.zeros(b,1,device=device))  # D(G(x)) should be 0
    loss_d.backward();
    opt_d.step()  # BUG
    z=torch.randn(b,z_dim,device=device);
    fake=Gnet(z.view(b,z_dim,1,1))
    loss_g=crit(Dnet(fake),
                torch.ones(b,1,device=device))  # BUG non-sat
    loss_g.backward()  # BUG missing step
    opt_d.step()
print('Now fix all the issues.')

  0%|          | 0/235 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 235/235 [03:18<00:00,  1.18it/s]

Now fix all the issues.
